In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [4]:
%cd /content/gdrive/MyDrive/PCB_QM

/content/gdrive/MyDrive/PCB_QM


In [3]:
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd

In [18]:
import pickle

In [22]:
csv_data = pd.read_csv('PCB_LABELING.csv')
prod1_data = csv_data[csv_data['prod_label'] == 1]
prod1_data.shape

(121, 4)

In [23]:
img_paths = prod1_data.path
labels = prod1_data.error_label

In [24]:
normal_img_paths = img_paths[labels == 0]
fault_img_paths = img_paths[labels != 0]

In [32]:
len(fault_img_paths)

120

In [25]:
fault_label = labels[labels != 0]

In [52]:
def create_patches_with_positions(image, patch_size=(64, 64)):
    patches = []
    positions = []
    for i in range(0, image.shape[0] - patch_size[0] + 1, patch_size[0]):
        for j in range(0, image.shape[1] - patch_size[1] + 1, patch_size[1]):
            patch = image[i:i+patch_size[0], j:j+patch_size[1]]
            patches.append(patch)
            positions.append((i, j))
    return np.array(patches), positions

In [71]:
def split_image(image, block_size=(64, 64)):
    # 이미지의 높이와 너비 가져오기
    h, w = image.shape[:2]
    # 블록 크기
    bh, bw = block_size
    # 블록으로 이미지 분할
    blocks = []
    for y in range(0, h, bh):
        for x in range(0, w, bw):
            block = image[y:y+bh, x:x+bw]
            blocks.append((block, (y, x)))
    return blocks

In [62]:
normal_image = cv2.imread(normal_img_paths[0])  # 하나의 정상 이미지 선택

In [73]:
# 패치 생성
block_size = (256, 256)
noraml_blocks = split_image(normal_image, block_size)


In [74]:
def compare_blocks_psnr(block1, block2, threshold=30):
    # 두 블록의 평균 절대 차이를 계산
    diff = np.abs(block1.astype(np.int32) - block2.astype(np.int32))
    psnr = cv2.PSNR(block1, block2)
    return psnr >= threshold

In [75]:
def compare_images(image1, image2, block_size=(64, 64), threshold=30):
    blocks1 = split_image(image1, block_size)
    blocks2 = split_image(image2, block_size)

    if len(blocks1) != len(blocks2):
        raise ValueError("The images do not have the same number of blocks")

    different_blocks = []
    for (block1, pos1), (block2, pos2) in zip(blocks1, blocks2):
        if not compare_blocks_psnr(block1, block2, threshold):
            different_blocks.append((block1, block2, pos1))

    return different_blocks

In [76]:
def plot_different_blocks(different_blocks):
    num_blocks = len(different_blocks)
    fig, axes = plt.subplots(num_blocks, 2, figsize=(8, num_blocks * 4))

    # If there is only one block, axes will not be a 2D array, so we need to make it so
    if num_blocks == 1:
        axes = np.expand_dims(axes, axis=0)

    for i, (block1, block2, pos) in enumerate(different_blocks):
        axes[i, 0].imshow(cv2.cvtColor(block1, cv2.COLOR_BGR2RGB))
        axes[i, 0].set_title(f'Original Block at {pos}')
        axes[i, 0].axis('off')

        axes[i, 1].imshow(cv2.cvtColor(block2, cv2.COLOR_BGR2RGB))
        axes[i, 1].set_title(f'Altered Block at {pos}')
        axes[i, 1].axis('off')

    plt.tight_layout()
    plt.show()

In [77]:
threshold = 40

In [82]:
fault_img_paths.iloc[1].split('/')[4].split('.')[0]
_,_,_,folder_name,file_name = fault_img_paths.iloc[1].split('/')

In [84]:
file_name

'01_missing_hole_02.jpg'

In [88]:
def save_image(different_blocks,folder,filename):
    num_blocks = len(different_blocks)

    for i, (block1, block2, pos) in enumerate(different_blocks):
        save_img = cv2.cvtColor(block2, cv2.COLOR_BGR2RGB)

        new_filename = f"{filename}_{i}.jpg"
        new_image_path = os.path.join(folder, new_filename)
        cv2.imwrite(new_image_path, save_img)


In [89]:
for img_path in fault_img_paths:
    fault_img = cv2.imread(img_path)
    _,_,_,folder_name,file_name = img_path.split('/')

    full_folder = f'./Patch_Image/{folder_name}/'
    file_name = file_name.split('.')[0]
    # 저장할 디렉토리가 없으면 생성
    os.makedirs(full_folder, exist_ok=True)

    # 이미지 크기가 같아야 함
    if normal_image.shape != fault_img.shape:
        raise ValueError("The images must have the same dimensions")

    # 이미지 비교
    different_blocks = compare_images(normal_image, fault_img, block_size=block_size,threshold= threshold)

    save_image(different_blocks,full_folder,file_name)

In [90]:
def save_gray_image(different_blocks,folder,filename):
    num_blocks = len(different_blocks)

    for i, (block1, block2, pos) in enumerate(different_blocks):
        save_img = cv2.cvtColor(block2, cv2.COLOR_BGR2GRAY)

        new_filename = f"{filename}_{i}.jpg"
        new_image_path = os.path.join(folder, new_filename)
        cv2.imwrite(new_image_path, save_img)

In [91]:
for img_path in fault_img_paths:
    fault_img = cv2.imread(img_path)
    _,_,_,folder_name,file_name = img_path.split('/')

    full_folder = f'./Patch_Gray_Image/{folder_name}/'
    file_name = file_name.split('.')[0]
    # 저장할 디렉토리가 없으면 생성
    os.makedirs(full_folder, exist_ok=True)

    # 이미지 크기가 같아야 함
    if normal_image.shape != fault_img.shape:
        raise ValueError("The images must have the same dimensions")

    # 이미지 비교
    different_blocks = compare_images(normal_image, fault_img, block_size=block_size,threshold= threshold)

    save_gray_image(different_blocks,full_folder,file_name)